In [1]:
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz

## Hulu Merge

#### Goals:

- Merge Hulu tv shows with IMDB ID


## 1. Import imdb and hulu data

In [2]:
imdb = pd.read_pickle("../Data/imdb_tv_all.pkl")

hulu = pd.read_csv("Data_Hulu_Disney/hulu_shows.csv")

We will look at how many titles we have using the shape function, which will return a tuple indicating number of rows and number of columns.

In [3]:
hulu.shape

(1744, 5)

## 2. Merge dataframes

We will start by merging on title and year in order to get the right IMDB ID (tconst) because there might be some remakes.
We will do this by using a left join in order to keep all the original hulu titles.

In [4]:
hulu_genres = hulu.merge(imdb, left_on=["show", "year"], right_on=["originalTitle", "startYear"], how="left")

From the new merged data frame, we will get a new data frame called hulu_missing1 to get the missing titles that didn't find a match.

In [5]:
hulu_missing1 = hulu_genres[hulu_genres["tconst"].isna()].reset_index(drop=True)[["show", "year", "rating"
                                                                                           , "imdb", "rotten_tomatoes"]]

In [6]:
hulu_missing1.shape

(526, 5)

We are missing 526 titles from 1744 in total.

We will now merge but now using primaryTitle instead of originalTitle, since some of the titles might be in their original language.

In [7]:
hulu_missing1 = hulu_missing1.merge(imdb, left_on=["show", "year"], right_on=["primaryTitle", "startYear"], how="left")

We will create another data frame for those titles who weren't matched.

In [8]:
hulu_missing2 = hulu_missing1[hulu_missing1["tconst"].isna()].reset_index(drop=True)[["show", "year", "rating"
                                                                                               , "imdb", "rotten_tomatoes"]]

In [9]:
hulu_missing2.shape

(416, 5)

We are now missing 416 titles. Since we don't have something else to relate to in this dataframe, we will now import the other data frame from the IMDB data base which includes all different titles created for a single one and the IMDB ID (titleID).

In [10]:
title_regions = pd.read_csv("C:/Users/Admin/Documents/ironhack/title.akas.tsv.gz", sep="\t", low_memory=False)

In [11]:
title_regions.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,1,Карменсіта,UA,\N,imdbDisplay,\N,0
1,tt0000001,2,Carmencita,DE,\N,\N,literal title,0
2,tt0000001,3,Carmencita - spanyol tánc,HU,\N,imdbDisplay,\N,0
3,tt0000001,4,Καρμενσίτα,GR,\N,imdbDisplay,\N,0
4,tt0000001,5,Карменсита,RU,\N,imdbDisplay,\N,0


We will do the merge of the missing titles with the new title_regions data frame by title name.

In [12]:
hulu_missing2 = hulu_missing2.merge(title_regions, left_on=["show"], right_on=["title"], how="left")

In [13]:
hulu_missing3 = hulu_missing2[hulu_missing2["titleId"].isna()].reset_index(drop=True)[["show", "year", "rating"
                                                                                                , "imdb", "rotten_tomatoes"]]

In [14]:
hulu_missing3.shape

(219, 5)

Now we are just missing 219 titles. We will now try to find the missing title IDs with fuzzy wuzzy.

### 3. Find missing titles using fuzzywuzzy

We will first create a list for missing shows.

In [15]:
hulu_shows_missing = [show for show in hulu_missing3["show"]]

In [16]:
imdb_titles = [title for title in imdb["primaryTitle"]]

We will use the same function we created on 03.2_a_netflix_missing_title_ids to find the most similar title on the imdb title basics file.

In [17]:
def find_shows(show):
    matches = []

    for title in imdb_titles:
        # compute ratio and remove case-sensitivity
        ratio = fuzz.ratio(title.lower(), show.lower())

        # add all matches to list with ratio > 60
        if ratio >= 60:
            matches.append((title, show, ratio))
    
    # return none if there was no match found
    if len(matches) == 0:
        return None
    return sorted(matches, key=lambda x: x[2], reverse=True)[0][0]

In [18]:
hulu_missing3["imdb_titles"] = hulu_missing3["show"].apply(lambda x: find_shows(x))

In [19]:
hulu_missing3.head()

,show,year,rating,imdb,rotten_tomatoes,imdb_titles
0,Naruto Shippūden,2007,7+,8.6,NaN,Naruto: Shippûden
1,Peep show,2003,18+,8.6,96%,Peep Show
2,DAVE,2020,18+,8.4,62%,Dave
3,Food Wars! Shokugeki no Soma,2015,18+,8.3,NaN,None
4,Nathan For You,2013,16+,8.8,97%,Nathan for You


From the first 10 missing titles we can see that our function did a good job, we will now take a look at the titles that weren't found a match.

In [20]:
hulu_missing3[hulu_missing3["imdb_titles"].isna()]

,show,year,rating,imdb,rotten_tomatoes,imdb_titles
3,Food Wars! Shokugeki no Soma,2015,18+,8.3,NaN,None
59,Kono Yuusha ga Ore Tueee Kuse ni Shinchou Sugiru,2019,NaN,7.4,NaN,None
60,Hakuoki -Demon Of The Fleeting Blossom-,2010,18+,7.4,NaN,None
83,恋と選挙とチョコレート,2012,18+,7.2,NaN,None
105,To LOVEる -とらぶる- ダークネス2nd,2012,16+,7.1,NaN,None
165,Ryan's World Specials presented by pocket.watch,2019,NaN,NaN,NaN,None


Since we have only 6 titles, we will remove them from our data.

## 4. Merge IMDB IDs with missing titles

We will still merge on the year, in order to be sure that we are not merging a remake or a similar title.

In [21]:
hulu_missing3 = hulu_missing3.merge(imdb, how="left", left_on=["imdb_titles", "year"], right_on=["primaryTitle", "startYear"])

In [22]:
hulu_missing3.head()

,show,year,rating,imdb,rotten_tomatoes,imdb_titles,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,Naruto Shippūden,2007,7+,8.6,NaN,Naruto: Shippûden,tt0988824,tvSeries,Naruto: Shippûden,Naruto: Shippûden,0.0,2007.0,2007.0,24,"Action,Adventure,Animation"
1,Peep show,2003,18+,8.6,96%,Peep Show,tt0387764,tvSeries,Peep Show,Peep Show,0.0,2003.0,2003.0,25,Comedy
2,DAVE,2020,18+,8.4,62%,Dave,tt8531222,tvSeries,Dave,Dave,0.0,2020.0,2020.0,30,Comedy
3,Food Wars! Shokugeki no Soma,2015,18+,8.3,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Nathan For You,2013,16+,8.8,97%,Nathan for You,tt2297757,tvSeries,Nathan for You,Nathan for You,0.0,2013.0,2013.0,30,"Comedy,Documentary"


We will now take a look at the titles who didn't find a match.

In [23]:
hulu_missing3[hulu_missing3["tconst"].isna()]

,show,year,rating,imdb,rotten_tomatoes,imdb_titles,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
3,Food Wars! Shokugeki no Soma,2015,18+,8.3,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,Crayon Shin-chan,1992,7+,7.9,NaN,Shinchan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14,Monogatari,2009,16+,8.0,NaN,Ai monogatari,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19,Land of Honor,2014,16+,7.9,NaN,Blade of Honor,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29,WWE Friday Night SmackDown,1999,7+,7.3,NaN,Friday Night Rock Show,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
211,Real Estate with Rosanna,2017,NaN,NaN,NaN,Real Estate with Ramon,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
213,On This Day in Black History Month,2019,NaN,NaN,NaN,NBC News Presents: Black History Month,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
214,The Big Vote,2018,NaN,NaN,NaN,The Big One,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
217,The Paley Center,2000,NaN,NaN,NaN,At the Paley Center,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


We can see that most of this titles don't have a similar imdb titles, due to this, we will drop this columns also.

In [24]:
hulu_missing_ids = hulu_missing3[~hulu_missing3["tconst"].isna()].reset_index(drop=True).drop_duplicates("tconst")

Now we are just missing 110 titles. We will remove this titles for our project.

## 5. Join data frames with IMDB IDs

We will join all data frames without null values to get a final hulu data frame containing the IMDB ID.
We will drop duplicates because the title regions may have joined multiple rows since some titles have the same name in different regions.

In [25]:
hulu_genres = hulu_genres[~hulu_genres["tconst"].isna()]

In [26]:
hulu_genres = hulu_genres.append(hulu_missing1[~hulu_missing1["tconst"].isna()])

In [27]:
hulu_genres = hulu_genres.append(hulu_missing2[~hulu_missing2["titleId"].isna()])

In [28]:
hulu_genres = hulu_genres.append(hulu_missing_ids)

In [29]:
hulu_genres = hulu_genres.drop_duplicates("tconst")

In [30]:
hulu_genres = hulu_genres.reset_index(drop=True)

In [31]:
hulu_genres.head()

,show,year,rating,imdb,rotten_tomatoes,tconst,titleType,primaryTitle,originalTitle,isAdult,...,genres,titleId,ordering,title,region,language,types,attributes,isOriginalTitle,imdb_titles
0,Rick and Morty,2013,18+,9.2,94%,tt2861424,tvSeries,Rick and Morty,Rick and Morty,0.0,...,"Adventure,Animation,Comedy",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Fargo,2014,18+,8.9,96%,tt2802850,tvSeries,Fargo,Fargo,0.0,...,"Crime,Drama,Thriller",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Vikings,2013,18+,8.5,93%,tt2306299,tvSeries,Vikings,Vikings,0.0,...,"Action,Adventure,Drama",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Brooklyn Nine-Nine,2013,16+,8.4,97%,tt2467372,tvSeries,Brooklyn Nine-Nine,Brooklyn Nine-Nine,0.0,...,"Comedy,Crime",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Community,2009,7+,8.5,88%,tt1439629,tvSeries,Community,Community,0.0,...,Comedy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


We will now calculate the ratio of titles with an imdb id match and the original titles.

In [32]:
len(hulu_genres) / len(hulu)

0.8405963302752294

This means that we will be using 84% of the original data.

## 6. Cleaning final data

We will create a final data frame including:
- show
- year
- rating
- imdb
- rotten_tomatoes
- imdb_id
- all data from title basics

First, we will need to create the imdb_id column, this will have tconst and titleId values.

In [33]:
hulu_genres["imdb_id"] = np.where(hulu_genres["tconst"].isna(), hulu_genres["titleId"], hulu_genres["tconst"])

In [34]:
hulu_genres["imdb_id"].isna().value_counts()

False    1466
Name: imdb_id, dtype: int64

This means we have all the IMDB IDs for all the titles in this final data frame. We will now remove columns that are not show, year, rating, imdb, rotten_tomatoes and imdb_id and merge again.

In [35]:
to_drop = [col for col in hulu_genres.columns if col not in ["show", "year", "rating", "imdb", "rotten_tomatoes", "imdb_id"]]

In [36]:
hulu_genres = hulu_genres.drop(columns=to_drop)

In [37]:
hulu_genres.head()

,show,year,rating,imdb,rotten_tomatoes,imdb_id
0,Rick and Morty,2013,18+,9.2,94%,tt2861424
1,Fargo,2014,18+,8.9,96%,tt2802850
2,Vikings,2013,18+,8.5,93%,tt2306299
3,Brooklyn Nine-Nine,2013,16+,8.4,97%,tt2467372
4,Community,2009,7+,8.5,88%,tt1439629


Now we will merge all data from title basics

In [38]:
hulu_genres = hulu_genres.merge(imdb, how="left", left_on="imdb_id", right_on="tconst")

In [39]:
hulu_genres.head()

,show,year,rating,imdb,rotten_tomatoes,imdb_id,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,Rick and Morty,2013,18+,9.2,94%,tt2861424,tt2861424,tvSeries,Rick and Morty,Rick and Morty,0,2013.0,2013.0,23,"Adventure,Animation,Comedy"
1,Fargo,2014,18+,8.9,96%,tt2802850,tt2802850,tvSeries,Fargo,Fargo,0,2014.0,2014.0,53,"Crime,Drama,Thriller"
2,Vikings,2013,18+,8.5,93%,tt2306299,tt2306299,tvSeries,Vikings,Vikings,0,2013.0,2013.0,44,"Action,Adventure,Drama"
3,Brooklyn Nine-Nine,2013,16+,8.4,97%,tt2467372,tt2467372,tvSeries,Brooklyn Nine-Nine,Brooklyn Nine-Nine,0,2013.0,2013.0,22,"Comedy,Crime"
4,Community,2009,7+,8.5,88%,tt1439629,tt1439629,tvSeries,Community,Community,0,2009.0,2009.0,22,Comedy


In [40]:
hulu_genres = hulu_genres.drop(columns="tconst")

## 7. Check null and unique values

In [41]:
hulu_genres.isna().sum()

show                  0
year                  0
rating              332
imdb                123
rotten_tomatoes    1095
imdb_id               0
titleType             0
primaryTitle          0
originalTitle         0
isAdult               0
startYear             0
endYear               0
runtimeMinutes        0
genres                0
dtype: int64

In [42]:
hulu_genres.nunique(axis=0)

show               1425
year                 66
rating                4
imdb                 74
rotten_tomatoes      73
imdb_id            1466
titleType             4
primaryTitle       1426
originalTitle      1425
isAdult               1
startYear            66
endYear              66
runtimeMinutes       75
genres              277
dtype: int64

From the previous cells we can see that:
- Most values are missing for rotten_tomatoes.
- isAdult has just one value.

We will drop this two columns, since we cannot get much information from it.

In [43]:
hulu_genres = hulu_genres.drop(columns=["rotten_tomatoes", "isAdult"])

## 8. Change data types

- rating: We will remove the '+' sign and turn it into an integer
- runtimeMinutes: we will change the type to integer

#### i. rating

In [44]:
hulu_genres["rating"] = [str(i).replace("+", "") for i in hulu_genres["rating"]]

In [45]:
hulu_genres["rating"].value_counts()

16     463
nan    332
7      328
18     215
all    128
Name: rating, dtype: int64

Since rating is a string type, we will convert the 'nan' values to null and 'all' to 0, meaning that the series can be watched by all ages.

In [46]:
hulu_genres["rating"] = np.where(hulu_genres["rating"] == "nan", None, hulu_genres["rating"])
hulu_genres["rating"] = np.where(hulu_genres["rating"] == "all", 0, hulu_genres["rating"])

In [47]:
hulu_genres["rating"].value_counts()

16    463
7     328
18    215
0     128
Name: rating, dtype: int64

In [48]:
hulu_genres["rating"] = pd.to_numeric(hulu_genres["rating"], errors="coerce")

#### ii. runtimeMinutes

In [49]:
hulu_genres["runtimeMinutes"] = pd.to_numeric(hulu_genres["runtimeMinutes"], errors="coerce")

#### ii. Check final data types

In [50]:
hulu_genres.dtypes

show               object
year                int64
rating            float64
imdb              float64
imdb_id            object
titleType          object
primaryTitle       object
originalTitle      object
startYear         float64
endYear           float64
runtimeMinutes    float64
genres             object
dtype: object

## 9. Rename rating and imdb columns

In [51]:
hulu_genres = hulu_genres.rename(columns={"rating":"age", "imdb":"imdb_rating"})

## 10. Export data

In [52]:
# hulu_genres.to_pickle("Data_Hulu_Disney/hulu_final_clean.pkl")